**Students:**
Groupe 11
* Conet Brieuc (11181800)
* Deside Guillaume (39731800)


# Project 2 : Hidden Markov models and optimal control
Authors : Simon Vandergooten and Clémence Vandamme.

In this second project, you will implement a Hidden Markov Model with 3 states and apply to your posterior probabilities optimal control. In other words, based on your knowledge and uncertainties, you will decide the optimal actions you need to take in any situation.

## Context :
The belgian government is trying to develop a new strategy to differentiate between patients with hypertension and those with hypotension in order to help preventing fainting and cardiac arrest.

Patients can be in one of the three following states : healthy, hypertension or hypotension. The patient's health status is represented by the vector $S$, where each value $s_t$ represents the patient's status in day $t$. The government also provides you the transition probabilities between each state, based on data from the national public health department.

You received the blood pressure measurements of a patient for 150 days. Those measurements are made with a new state-of-the-art tool. However, he design of this tool is not yet perfected and the measurements are for the moment very imprecise. Based on these data, you are asked to provide an estimate of the true state of the patient. 

Furthermore, two drugs are available on the market. One aims to lower the blood pressure, the other increases it. At each moment, you need to determine if you should take a drug and which one. However, take into account that these drugs have a cost and that taking a drug when healthy represents a risk to make the situation worse.  

## Practical information:
### HMM

The following graph sums up the different states and their transition probabilities. **The initial state is healthy**.

<img alt='Solution hint' align='center' width=413 height=300 src=https://raw.githubusercontent.com/svandergoote/LGBIO2060-2021/master/Solutions/Projet2_bis.png> 



### Measurements

Concerning the measurements $m_t$, the systolic blood pressure levels are distributed as follow:

* $m_t \sim \mathcal{N}$(120, $\sigma_{healthy}^2$) if $s_t$ = 'healthy'.

* $m_t \sim \mathcal{N}$(160, $\sigma_{hyper}^2$) if $s_t$ = 'hyper'.

* $m_t \sim \mathcal{N}$(80, $\sigma_{hypo}^2$) if $s_t$ = 'hypo'.

Where $\sigma_{healthy}, \, \sigma_{hyper}, \, \sigma_{hypo}=27, \, 30\, , 28$  are the measurement noise related to the new tool.
 


### Potential actions and their effects

Each day, you have 3 options:
- Take a drug A
    - It has 80% chances to **lower** your tension to 120 if you suffer from **hypertension**. (Therefore, 20% chances to have no effect). 
    - It has 40% chance to **lower** your tension to 80 if you were **healthy**. 
    - It has **no effect** if you suffer from **hypotension**.

- Take a drug B 
    - It has 80% chances to **increase** your tension to 120 if you suffer from **hypotension**.
    - It has 40% chances to **increase** your tension to 160 if you were **healthy**.
    - It has **no effect** if you suffer from **hypertension**.

- Do nothing: no effect on your blood pressure in any state. 

NOTE : The action has no impact on the state transition and the state itself. It should only be the most appropropriate action based on your belief about the state.


### Utility and cost
Utility values associated to blood pressure: 
- 120 mmHg : U = +2
- 160 mmHg : U = -2 (risk of heart attack)
- 80 mmHg : U = -1 (risk of fainting) 

Costs:
- Drug A : 2
- Drug B : 2







## Guidelines 

**READ THIS PART CAREFULLY**

For the first part of the project, you are asked to estimate the posterior probability of each state at any time, based on the measurements vector M and on the transition probabilities. M contains 150 measurements. Then, graphically represent the evolution of these probabilities. The way you plot these data are up to you, make it readable and interpretable (you do not necessarily have to represent all the time step). 


In the second part, determine the policy you will follow at each time step to choose the optimal action. Based on this policy, return the vector of actions taken for the given data. A policy must be optimal in the sense that it maximizes the benefit (utility) and minimizes the cost. It simply consists in setting a threshold on your posterior belief. For example, "*if I have more than 65% probability to have hypertension, I will take drug A*" is a policy. We give you the 100 first true states to test different policies (i.e, different thresholds) and determine which one is the best. Indeed, with the true states, you can determine the impact of your actions and compute both the benefit and the cost. 
After finding your optimal policy, you can look at the actions chosen for your measures for which you do not have the true states (i.e the last 50 measurements).


Finally, discuss the impact of some relevant parameters of the model. For exemple, what do you observe in the policy and selected actions if the cost of both drugs increase ? 



### To sum up:

1) Create your HMM and compute the posterior probabilities associated to each state based on the measurements. 

2) Graphically represent the evolution of the posteriors.

3) Define an optimal policy (this answer must appear clearly).  

4) Based on your policy, determine the drug to take at each time step for the given data. 

5) Discuss the impact of the parameters of the model on the policy and the resulting actions. 


### Data:

- The vector $S$ containing the 100 first hidden states $s_t$: 
  * $s_t$ = 'healthy' if the patient's state is healthy for day $t$ 
  * $s_t$ = 'hyper' if the patient's state is hypertension for day $t$ 
  * $s_t$ = 'hypo' if the patient's state is hypotension for day $t$ 

  It must be only used to determine the optimal policy. **It cannot be used for the HMM**. 

- The vector $M$ containing 150 measurements $m_t$. It is a vector of scalars. 

### Submission
Like for the first project, you must submit your notebook with all your answers. \\
The **deadline is the Thurday 25 November 23:59**. Name your notebook as follow : "LGBIO2060_Projet2_Grxx". \\
Don't forget to register for a timeslot for the oral evaluation.

## General remarks

⚠ Some functions were adapted from the LGBIO2060-modeling of biological systems 4th practical session. The plotly package was utilized for the visualization since it is more powerful for data analysis. One disadvantage of the plotly library is that graphs do not persist on jupyter (or google) notebook, therefore we must relaunch to see graphs when we start a new session.

## Code

### Import

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import ipywidgets as widgets   
from ipywidgets import interactive, interact, HBox, Layout,VBox

### General informations

In [2]:
nb_measurement = 150
nb_true_state = 100

# useful for some functions
# index of states in lists
dico_index = {
    0 : "healthy",
    1 : "hypo",
    2 : "hyper"
}

initial_probability = np.array([1., 0., 0.])  # [healthy,hypo,hyper]

lst_means = np.array([120, 80, 160]) # [healthy,hypo,hyper]
lst_variances = np.array([27 * 27, 28 * 28, 30 * 30]) # [healthy,hypo,hyper]

dico_utility = {
    "healthy": 2,
    "hypo": -1,
    "hyper": -2
}

#probabilities to be healthy if you take the drug depending on the state
dico_A = {  
    "hyper": 0.8,
    "healthy": 0.6,
    "hypo": 0.0
}

dico_B = {
    "hypo": 0.8,
    "healthy": 0.6,
    "hyper": 0.0
}

#cost of different actions
dico_cost = { 
    "A": 2,
    "B": 2,
    "/": 0 # do nothing
}

transition_matrix = np.array([[0.3, 0.3, 0.4], #[healthy,hypo,hyper]
                              [0.6, 0.3, 0.1], 
                              [0.25, 0.1, 0.65]])

In [3]:
Measurement = pd.read_csv("Datasets/M.csv", header=None)
Measurement = Measurement[1].values[1:]

In [4]:
S_training = pd.read_csv("Datasets/S_training.csv", header=None)
S_training = S_training[1].values[1:]

### Calculate posterior probabilities

#### Fonctions

In [5]:
def compute_likelihood(means, variances, M):
    """
    Adapted from LGBIO2060 - TP4

    Calculate likelihood of seeing data `M` for all measurement models

    Args:
      means (numpy array): Mean measurement for each state [healthy,hypo,hyper].
      variances (numpy array): Variance measurement for each state [healthy,hypo,hyper].
      M (float or numpy vector) : Measurements

    Returns:
      L (numpy vector or matrix): the likelihood. if M is a float -> L = [p(M|s=1), p(M|s=-1)]
                                                   M is a vector-> L = [[p(M|s=1)],
    """
    p_healthy = stats.norm(means[0], np.sqrt(variances[0]))
    p_hypo = stats.norm(means[1], np.sqrt(variances[1]))
    p_hyper = stats.norm(means[2], np.sqrt(variances[2]))

    L = np.array([p_healthy.pdf(M), p_hypo.pdf(M), p_hyper.pdf(M)])
    return L

def one_step_update(T, posterior_tm1, M_t, means, variances):
    """
    Adapted from LGBIO2060 - TP4

    Given a HMM model, calculate the one-time-step updates to the posterior.

    Args:
        T (numpy array): transition matrix of the HMM
        posterior_tm1 (numpy array): Posterior at `t-1`
        M_t (numpy array): measurement at `t`
        means (numpy array): Mean measurement for each state [healthy,hypo,hyper].
        variances (numpy array): Variance measurement for each state [healthy,hypo,hyper].
    Returns:
        prediction (numpy array): prediction at `t` (Today's prior)
        likelihood (numpy array): likelihood of seeing data M_t for all measurements models
        posterior_t (numpy array): Posterior at `t`
    """
    # Calculate predictive probabilities (prior)
    prediction = T @ posterior_tm1

    # Get the likelihood (Hint: Use compute_likelihood)
    likelihood = compute_likelihood(means, variances, M_t)

    # Calculate posterior
    posterior_t = prediction * likelihood

    # Normalize
    posterior_t /= posterior_t.sum()

    return prediction, likelihood, posterior_t


def simulate_forward_inference(means, variances, T, start_proba, N, data):
    """
    Adapted from LGBIO2060 - TP4

    Given the HMM model, calculate posterior marginal predictions of x_t for N-1 time steps ahead based on
    evidence `data`.

    Args:
        means (numpy array): Mean measurement for each state [healthy,hypo,hyper].
        variances (numpy array): Variance measurement for each state [healthy,hypo,hyper].
        T (numpy array): transition matrix of the HMM
        start_proba (numpy array): initial probabilities
        N (int): length of returned array
        data (numpy array): measurements vector


    Returns:
        predictive_probs (numpy array): predictive probabilities
        likelihoods (numpy array): likelihood of seeing data M_t for all measurements models
        posterior_probs (numpy array): posterior probabilities
    """

    # Initialize arrays
    predictive_probs = np.zeros((N, 3))
    likelihoods = np.zeros((N, 3))
    posterior_probs = np.zeros((N, 3))

    # Calculate marginal for each latent state x_t
    # Start with the first element
    predictive_probs[0, :] = start_proba
    likelihoods[0, :] = compute_likelihood(means, variances, data[0])
    posterior = predictive_probs[0, :] * likelihoods[0, :]
    posterior /= np.sum(posterior)
    posterior_probs[0, :] = posterior

    # Then iterate for the rest of the N elements
    for t in range(1, N):
        prediction, likelihood, posterior = one_step_update(T, posterior_probs[t - 1, :], data[t], 
                                                                                            means, variances)
        # normalize and add to the list
        posterior /= np.sum(posterior)
        predictive_probs[t, :] = prediction
        likelihoods[t, :] = likelihood
        posterior_probs[t, :] = posterior

    return predictive_probs, likelihoods, posterior_probs

#### Calculation of posterior probabilities

In [6]:
predictive_probs, likelihoods, posterior_probs = simulate_forward_inference(lst_means, lst_variances, 
                                                                            transition_matrix,
                                                                            initial_probability, nb_measurement,
                                                                            Measurement)
#print(posterior_probs)  

#### Visualization

In [7]:
plot_state = np.zeros(nb_measurement)
prob_stat = np.zeros(nb_measurement)

plot_x_axis = np.arange(1, 151)

for i in range(nb_measurement):
    plot_state[i] = np.argmax(posterior_probs[i])
    prob_stat[i] = np.around((np.max(posterior_probs[i])),4)

In [9]:
trace1 = go.Scatter(
    x= plot_x_axis,
  y= posterior_probs[:,0],
  line={"shape": 'hv'},
  mode= 'lines',
  name= 'Healthy',
  marker_color= "green",
)

trace2 = go.Scatter(
    x= plot_x_axis,
  y= posterior_probs[:,1],
  line={"shape": 'hv'},
  mode= 'lines',
  name= 'Hypotension',
  marker_color= "blue",
)

trace3 = go.Scatter(
    x= plot_x_axis,
  y= posterior_probs[:,2],
  line={"shape": 'hv'},
  mode= 'lines',
  name= 'Hypertension',
  marker_color= "red",
)

fig = go.Figure(data=[trace1])
fig.update_yaxes(range=[-0.1,1.1])
fig.update_xaxes(range =[0,150])
fig.update_yaxes(
    ticktext=[0, 1],
    tickvals=[0, 1],
)

fig.update_layout(
    title="Posterior Probabilities : Healthy",
    xaxis_title="day",
    yaxis_title="Probability",
    font=dict(
        family="Times New Roman",
        size=16,
        color="green"
    )
)
fig.show()


fig1 = go.Figure(data=[trace2])
fig1.update_yaxes(range=[-0.1,1.1])
fig1.update_xaxes(range =[0,150])
fig1.update_yaxes(
    ticktext=[0, 1],
    tickvals=[0, 1],
)

fig1.update_layout(
    title="Posterior Probabilities : Hypotension",
    xaxis_title="day",
    yaxis_title="Probability",
    font=dict(
        family="Times New Roman",
        size=16,
        color="blue"
    )
)
fig1.show()

fig2 = go.Figure(data=[trace3])

fig2.update_yaxes(range=[-0.1,1.1])
fig2.update_xaxes(range =[0,150])
fig2.update_yaxes(
    ticktext=[0, 1],
    tickvals=[0, 1],
)

fig2.update_layout(
    title="Posterior Probabilities : Hypertension",
    xaxis_title="day",
    yaxis_title="Probability",
    font=dict(
        family="Times New Roman",
        size=16,
        color="red"
    )
)
fig2.show()

In [12]:
trace4 = go.Scatter(
    x= plot_x_axis,
  y= plot_state,
  line={"shape": 'hv'},
  mode= 'lines',
  name= 'state',
  marker_color= "orange",
)
fig4 = go.Figure(data=[trace4])
fig4.update_yaxes(range=[-0.1,2.1])
fig4.update_xaxes(range =[0,153])
fig4.update_yaxes(
    ticktext=["Healthy","Hypo","Hyper"],
    tickvals=[0, 1,2],
)
fig4.update_layout(
    title="State with greatest probability for each day",
    xaxis_title="day",
    yaxis_title="State",
    font=dict(
        family="Times New Roman",
        size=16,
        color="orange"
    )
)
fig4.show()

In [14]:
#Adapted from https://plotly.com/python/annotated-heatmap/
colorscale=[[0.0, 'rgb(255,255,255)'], [.2, 'rgb(255, 255, 153)'],
            [.4, 'rgb(153, 255, 204)'], [.6, 'rgb(179, 217, 255)'],
            [.8, 'rgb(240, 179, 255)'],[1.0, 'rgb(255, 77, 148)']]

z = np.reshape(plot_state,(10,15))/2
symbol = np.reshape(prob_stat,(10,15))

hover = []
for i in range(10):
    hoverbis = []
    for j in range(15):
        hoverbis.append("day " + str(i*15+j+1) + ":" + dico_index[z[i,j]*2])
    hover.append(hoverbis)



fig5 = ff.create_annotated_heatmap(z, annotation_text=symbol,colorscale=colorscale,text=hover,
                                                                       font_colors=['black'], hoverinfo='text')
fig5.update_layout(title_text='State with greatest probability for each day')
fig5.show()

### Choice of a policy

Each drug (A and B) will have its own threshold indicating whether the drug should be taken according to the posterior probailty of the state.
To determine the optimal thresholds and hence maximize utility, we experimented with several thresholds on posterior probability ranging from 0.3 to 1.0.

**Steps apllied on each pair of thresholds:**
    <ol>
      <li>Create a list of actions depending on two thresholds</li>
      <li>Calculate cost of this list</li>
      <li>Calculate utility based on True state</li>
      <li>Substract of cost from utility</li>
      <li>Compare with previous calculated utilities</li>
    </ol>

#### Fonctions

In [16]:
def wish_action(posterior, threshold_A, threshold_B):
    """
    Returns a list of actions depending on the two thresholds
    
    :param posterior: (numpy array) list of posterior probability
    :param threshold_A: (float) Threshold to take drug A
    :param threshold_B: (float) Threshold to take drug B
    :return: (numpy) array of actions
    """
    action = np.zeros(len(posterior), dtype=str)
    action[0] = "/"
    for i in range(1, len(posterior)):
        idx = np.argmax(posterior[i])
        if idx == 1 and posterior[i][idx] >= threshold_B:
            action[i] = "B"
        elif idx == 2 and posterior[i][idx] >= threshold_A:
            action[i] = "A"
        else:
            action[i] = "/"  # no drug taken
    return action


def cost_action(action):
    """
    Returns the cost of a actions'list

    :param action: (numpy array) list of actions
    :return: (int) cost of the list of actions
    """
    cost = 0
    for i in range(len(action)):
        cost += dico_cost[action[i]]
    return cost

def calculate_utility(S_true, action):
    """
    Returns utility depending on True state and list of actions

    :param S_true: (numpy array) True state
    :param action: (numpy array) Actions made
    :return: (int) the utility
    """
    utility = 0
    for i in range(len(S_true)):
        if action[i] == "/":
            utility += dico_utility[S_true[i]]
        elif action[i] == "A":
            if S_true[i] == "hypo":
                utility += dico_utility[S_true[i]]
            elif S_true[i] == "hyper":
                utility += dico_A["hyper"] * dico_utility["healthy"] + (1 - dico_A["hyper"])\
                                                                                    * dico_utility["hyper"]
            else:
                utility += (1-dico_A["healthy"]) * dico_utility["hypo"] + (dico_A["healthy"])\
                                                                                    * dico_utility["healthy"]
        else:
            if S_true[i] == "hyper":
                utility += dico_utility[S_true[i]]
            elif S_true[i] == "hypo":
                utility += dico_B["hypo"] * dico_utility["healthy"] + (1 - dico_B["hypo"])\
                                                                                    * dico_utility["hypo"]
            else:
                utility += (1-dico_B["healthy"]) * dico_utility["hyper"] + (dico_B["healthy"])\
                                                                                    * dico_utility["healthy"]
    return utility

#### Calculation of best thresholds

In [17]:
#List of differents thresholds
lst_threshold_A = np.arange(0.3,1.05,0.05)
lst_threshold_B = np.arange(0.3,1.005,0.05)

In [19]:
#To calculate cost
lst_cost = np.zeros((len(lst_threshold_A),len(lst_threshold_B)))
plot_threshold_A = np.zeros(len(lst_threshold_A)*len(lst_threshold_B))
plot_threshold_B = np.zeros(len(lst_threshold_A)*len(lst_threshold_B))

for i in range(len(lst_threshold_A)):
    for j in range(len(lst_threshold_B)):
        plot_threshold_A[i * len(lst_threshold_B) + j] = lst_threshold_A[i]
        plot_threshold_B[i * len(lst_threshold_B) + j] = lst_threshold_B[j]
        action =  wish_action(posterior_probs[0:nb_true_state], lst_threshold_A[i], lst_threshold_B[j])
        lst_cost[i,j] = cost_action(action)

In [20]:
#To calculate utility - cost
lst_utility_cost = np.zeros((len(lst_threshold_A),len(lst_threshold_B)))

for i in range(len(lst_threshold_A)):
    for j in range(len(lst_threshold_B)):
        actions = wish_action(posterior_probs[0:nb_true_state],lst_threshold_A[i],lst_threshold_B[j])
        lst_utility_cost[i,j] = calculate_utility(S_training,actions)
        lst_utility_cost[i,j] -= lst_cost[i,j]

#### Visualization

In [21]:
fig6 = go.Figure(data=[go.Scatter3d(
    x=plot_threshold_A, y=plot_threshold_B, z=lst_cost.flatten(),
    mode='markers',
    marker=dict(
        size=12,
        color=lst_cost.flatten(),                
        colorscale='Inferno',   
        opacity=0.6
    )
)])

fig6.update_layout(scene = dict(
                    xaxis_title='Threshold A',
                    yaxis_title='Threshold B',
                    zaxis_title='Cost'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=5))

fig6.update_layout(
    title={
        'text': "Cost for pair of thresholds",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig6.show()

We can see in the figure above that the lower the threshold, the more drugs will be used, and therefore the price will rise and vice-versa.

In [22]:
fig7 = go.Figure(data=[go.Scatter3d(
    x=plot_threshold_A, y=plot_threshold_B, z=lst_utility_cost.flatten(),
    mode='markers',
    marker=dict(
        size=12,
        color=lst_utility_cost.flatten(),                
        colorscale='Inferno',  
        opacity=0.6
    )
)])

fig7.update_layout(scene = dict(
                    xaxis_title='Threshold A',
                    yaxis_title='Threshold B',
                    zaxis_title='Utility and cost'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))

fig7.update_layout(
    title={
        'text': "Utility and cost for pair of thresholds",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})


fig7.show()

In [28]:
max_xy = np.where(lst_utility_cost == np.max(lst_utility_cost) )
idx = np.argmax(lst_utility_cost)
best_threshold_A = np.around(plot_threshold_A[idx],3)
best_threshold_B = np.around(plot_threshold_B[idx],3)

print("****** Obtained Values ******")
print("")
print("Threshold for drug A : " + str(np.around(plot_threshold_A[idx],3)))
print("")
print("Threshold for drug B : " + str(np.around(plot_threshold_B[idx],3)))
print("")
print("Max utility-cost : " + str(np.around(lst_utility_cost[max_xy][0],3)))
print("")
print("We must understand this value as follows : if I have more than {}% probability to have hypertension, I will take drug A or \
if I have more than {}% probability to have hypertension, I will take drug B.".format(\
                                                                     np.around(plot_threshold_A[idx],3)*100,\
                                                                     np.around(plot_threshold_B[idx],3)*100))

****** Obtained Values ******

Threshold for drug A : 0.65

Threshold for drug B : 0.75

Max utility-cost : -41.0

We must understand this value as follows : if I have more than 65.0% probability to have hypertension, I will take drug A or if I have more than 75.0% probability to have hypertension, I will take drug B.


### List of actions for 50 last days

In [29]:
action_end = wish_action(posterior_probs[100:150],best_threshold_A,best_threshold_B)
#for num_day in range(101,151):
    #print("day "+str(num_day) + ":" + str(action_end[num_day-101]))

In [30]:
#Adapted from https://plotly.com/python/annotated-heatmap/
colorscale=[[0.0, 'rgb(255,255,255)'], [.2, 'rgb(255, 255, 153)'],
            [.4, 'rgb(153, 255, 204)'], [.6, 'rgb(179, 217, 255)'],
            [.8, 'rgb(240, 179, 255)'],[1.0, 'rgb(255, 77, 148)']]

z = np.reshape(plot_state[100:150],(5,10))/2
symbol = np.reshape(action_end,(5,10))


hover = []
for i in range(5):
    hoverbis = []
    for j in range(10):
        hoverbis.append("day " + str(i*10+j+1+100) + ": " + str(prob_stat[100:150][i*10+j]))
    hover.append(hoverbis)



    
zbis = [[0,0.5,1.0]]
symbolbis = [["Healthy","Hypo","Hyper"]]
fig9 = ff.create_annotated_heatmap(zbis, annotation_text=symbolbis,colorscale=colorscale,
                                                 font_colors=['black'], hoverinfo='text')
fig9.update_layout(
    autosize=False,
    width=250,
    height=250,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    )
)
fig9.update_layout(title_text='Legend : State according to posterior probabilities')
fig9.show()

fig8 = ff.create_annotated_heatmap(z, annotation_text=symbol,colorscale=colorscale,
                                   text=hover,font_colors=['black'], hoverinfo='text')
fig8.update_layout(title_text='Action for the 50 last days')
fig8.show()




### Play with other parameters

#### Impact of the drugs' cost

In [32]:
cost_A_widget = widgets.IntSlider(0.9, description='cost A', min=1, max=4, step=1)
cost_B_widget = widgets.IntSlider(0.9, description='cost B', min=1, max=4, step=1)

@widgets.interact(
    cost_A = cost_A_widget,
    cost_B = cost_B_widget,
)
def print_ps(cost_A,cost_B):
    lst_threshold_A_bis = np.arange(0.3,1.05,0.05)
    lst_threshold_B_bis = np.arange(0.3,1.005,0.05)
    lst_utility_cost_bis = np.zeros((len(lst_threshold_A),len(lst_threshold_B)))

    for i in range(len(lst_threshold_A_bis)):
        for j in range(len(lst_threshold_B_bis)):
            actions = wish_action(posterior_probs[0:nb_true_state],lst_threshold_A_bis[i],
                                                                                        lst_threshold_B_bis[j])
            lst_utility_cost_bis[i,j] = calculate_utility(S_training,actions)
            cost = 0
            for z in range(nb_true_state):
                if (actions[z] == "A"):
                    cost += cost_A
                elif (actions[z] == "B"):
                    cost += cost_B
                else:
                    cost += 0
            lst_utility_cost_bis[i,j] -= cost
    print("Utility and cost :" + str(np.around(np.max(lst_utility_cost_bis),4)))
    idx = np.argmax(lst_utility_cost_bis)
    best_threshold_A = np.around(plot_threshold_A[idx],3)
    best_threshold_B = np.around(plot_threshold_B[idx],3)
    print("")
    print("Threshold for drug A : " + str(np.around(plot_threshold_A[idx],3)))
    print("")
    print("Threshold for drug B : " + str(np.around(plot_threshold_B[idx],3)))

    fig7 = go.Figure(data=[go.Scatter3d(
    x=plot_threshold_A, y=plot_threshold_B, z=lst_utility_cost_bis.flatten(),
    mode='markers',
    marker=dict(
        size=12,
        color=lst_utility_cost_bis.flatten(),                
        colorscale='Inferno',  
        opacity=0.6))])

    fig7.update_layout(scene = dict(
                    xaxis_title='Threshold A',
                    yaxis_title='Threshold B',
                    zaxis_title='Utility and cost'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))

    fig7.update_layout(title={
        'text': "Utility and cost for pair of thresholds",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})


    fig7.show()
    
    

interactive(children=(IntSlider(value=1, description='cost A', max=4, min=1), IntSlider(value=1, description='…

As we can observe above, the cost of drug A has a greater influence on its threshold than the drug B whereas they have the same efficiency. The threshold for the drug B starts at 0.75 for a cost of 1 and increases to 1 only when cost reaches 3.
The threshold for the drug A starts at 0.3 for a cost of 1 and increases to 1 only when cost reaches 4.
That can be explained by the fact that we want to prevent hypertension and drug A use decrease risk to remain in hypertension state that has a greater impact on utility than the hypotension state.

#### Impact of probability to be healthy if you take drugs

In [33]:
def calculate_utility(S_true, action,A_hyper_to_healthy,A_healthy_to_healthy,
                      B_hypo_to_healthy,B_healthy_to_healthy):
    """
    Returns utility depending on True state and list of actions
    :param S_true: (numpy array) True state
    :param action: (numpy array) Actions made
    :return: (int) the utility
    """
    utility = 0
    for i in range(len(S_true)):
        if action[i] == "/":
            utility += dico_utility[S_true[i]]
        elif action[i] == "A":
            if S_true[i] == "hypo":
                utility += dico_utility[S_true[i]]
            elif S_true[i] == "hyper":
                utility += A_hyper_to_healthy * dico_utility["healthy"] + (1 - A_hyper_to_healthy)\
                                                                                    * dico_utility["hyper"]
            else:
                utility += (1-A_healthy_to_healthy) * dico_utility["hypo"] + (A_healthy_to_healthy)\
                                                                                    * dico_utility["healthy"]
        else:
            if S_true[i] == "hyper":
                utility += dico_utility[S_true[i]]
            elif S_true[i] == "hypo":
                utility += B_hypo_to_healthy * dico_utility["healthy"] + (1 - B_hypo_to_healthy)\
                                                                                    * dico_utility["hypo"]
            else:
                utility += (1-B_healthy_to_healthy) * dico_utility["hyper"] + (B_healthy_to_healthy)\
                                                                                    * dico_utility["healthy"]
    return utility

In [35]:
drug_A_hyper_to_healthy_widget = widgets.FloatSlider(0.3, description='drug A:prob Hyper to healthy', min=0.5, 
                                                                                       max=1.0, step=0.1)
drug_A_hyper_to_healthy_widget.layout.width = '800px'
drug_A_healthy_to_healthy_widget = widgets.FloatSlider(0.9, description='drug A:prob healthy to healthy', 
                                                                                   min=0.5, max=1.0, step=0.1)
drug_A_healthy_to_healthy_widget.layout.width = '800px'
drug_B_hypo_to_healthy_widget = widgets.FloatSlider(0.9, description='drug B:prob Hypo to healthy', min=0.5,
                                                                                        max=1.0, step=0.1)
drug_B_hypo_to_healthy_widget.layout.width = '800px'
drug_B_healthy_to_healthy_widget = widgets.FloatSlider(0.9, description='drug B:prob Healthy to healthy', 
                                                                                   min=0.5, max=1.0, step=0.1)
drug_B_healthy_to_healthy_widget.layout.width = '800px'

@widgets.interact(
    A_hyper_to_healthy = drug_A_hyper_to_healthy_widget,
    A_healthy_to_healthy = drug_A_healthy_to_healthy_widget,
    B_hypo_to_healthy = drug_B_hypo_to_healthy_widget,
    B_healthy_to_healthy = drug_B_healthy_to_healthy_widget

)
def print_ps(A_hyper_to_healthy,A_healthy_to_healthy,B_hypo_to_healthy,B_healthy_to_healthy):
    lst_threshold_A_bis = np.arange(0.3,1.05,0.05)
    lst_threshold_B_bis = np.arange(0.3,1.005,0.05)
    lst_utility_cost_bis = np.zeros((len(lst_threshold_A),len(lst_threshold_B)))
    
    for i in range(len(lst_threshold_A_bis)):
        for j in range(len(lst_threshold_B_bis)):
            actions = wish_action(posterior_probs[0:nb_true_state],lst_threshold_A_bis[i],
                                                                                        lst_threshold_B_bis[j])
            lst_utility_cost_bis[i,j] = calculate_utility(S_training, actions,A_hyper_to_healthy,
                                                A_healthy_to_healthy,B_hypo_to_healthy,B_healthy_to_healthy)
            cost = 0
            for z in range(nb_true_state):
                if (actions[z] == "A"):
                    cost += 2
                elif (actions[z] == "B"):
                    cost += 2
                else:
                    cost += 0
            lst_utility_cost_bis[i,j] -= cost
    print("Utility and cost :" + str(np.around(np.max(lst_utility_cost_bis),4)))
    idx = np.argmax(lst_utility_cost_bis)
    best_threshold_A = np.around(plot_threshold_A[idx],3)
    best_threshold_B = np.around(plot_threshold_B[idx],3)
    print("")
    print("Threshold for drug A : " + str(np.around(plot_threshold_A[idx],3)))
    print("")
    print("Threshold for drug B : " + str(np.around(plot_threshold_B[idx],3)))

    fig7 = go.Figure(data=[go.Scatter3d(
    x=plot_threshold_A, y=plot_threshold_B, z=lst_utility_cost_bis.flatten(),
    mode='markers',
    marker=dict(
        size=12,
        color=lst_utility_cost_bis.flatten(),                
        colorscale='Inferno',  
        opacity=0.6))])

    fig7.update_layout(scene = dict(
                    xaxis_title='Threshold A',
                    yaxis_title='Threshold B',
                    zaxis_title='Utility and cost'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))

    fig7.update_layout(title={
        'text': "Utility and cost for pair of thresholds",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})


    fig7.show()
    

interactive(children=(FloatSlider(value=0.5, description='drug A:prob Hyper to healthy', layout=Layout(width='…

If we compare it with previous, the efficacy of the drugs have lower impact on thresholds than the cost. The utility changes but thresholds remain constant most of time. That can be explained by the influence of the efficacy of drugs on the utility's formula. 


As a result, we may infer that it is more necessary to lower drug costs (while maintaining efficacy) than to increase efficacy (with the same cost).